In [8]:
import pandas as pd
import numpy as np
from urllib import request
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

import math
import time

In [56]:
def find_review(): # name: 영화 이름으로 데이터프레임 dict 안에 넣음 / 메가박스용
    driver = webdriver.Chrome('chromedriver')
    driver.get(f_url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    result = []
    result_charm = []
    result2 = []
    result2_charm = []
    for i in range(10): # 10페이지 까지
        if (i!=0):
            btn = driver.find_element(By.CSS_SELECTOR,f'#contentData > div > div.movie-idv-story > nav > a:nth-child({i+1})')# 버튼 생성
            btn.click()# 클릭
            time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        page = soup.select_one('#contentData > div > div.movie-idv-story > ul')
        result.append(page.findAll('div',{'class':'story-txt'}))
        result_charm.append(page.findAll('div',{'class':'story-recommend'}))
        
    btn = driver.find_element(By.CSS_SELECTOR,'#contentData > div > div.movie-idv-story > nav > a.control.next') 
    btn.click()
    time.sleep(0.5)

    for i in range(11,math.ceil(int(driver.find_element(By.CSS_SELECTOR,'#cnt1').text.replace(',',''))/10)+1): # 총 개수 / 10 한 몫만큼의 페이지 존재, 검색 지금은 2~82페이지
        if (i%10==0 and i>0):
            btn = driver.find_element(By.CSS_SELECTOR,f'#contentData > div > div.movie-idv-story > nav > a:nth-child({12})')# 버튼 생성
            btn.click()# 클릭
            time.sleep(0.5)
            
            soup = BeautifulSoup(driver.page_source,'html.parser')
            page = soup.select_one('#contentData > div > div.movie-idv-story > ul')
            result2.append(page.findAll('div',{'class':'story-txt'}))
            result2_charm.append(page.findAll('div',{'class':'story-recommend'}))
            
            btn = driver.find_element(By.CSS_SELECTOR,'#contentData > div > div.movie-idv-story > nav > a.control.next') # 10도달 시 다음
            btn.click()
            time.sleep(0.5)
        elif (i%10!=1):
            btn = driver.find_element(By.CSS_SELECTOR,f'#contentData > div > div.movie-idv-story > nav > a:nth-child({i%10+2})')# 버튼 생성
            btn.click()# 클릭
            time.sleep(0.5)
            
            soup = BeautifulSoup(driver.page_source,'html.parser')
            page = soup.select_one('#contentData > div > div.movie-idv-story > ul')
            result2.append(page.findAll('div',{'class':'story-txt'}))
            result2_charm.append(page.findAll('div',{'class':'story-recommend'}))
        elif (i%10==1):
            soup = BeautifulSoup(driver.page_source,'html.parser')
            page = soup.select_one('#contentData > div > div.movie-idv-story > ul')
            result2.append(page.findAll('div',{'class':'story-txt'}))
            result2_charm.append(page.findAll('div',{'class':'story-recommend'}))
        
    result3 = result+result2 # 두 결과 합치기(댓글)
    result_charm3 = result_charm + result2_charm # 두 결과 합치기(매력)

    charm3= []
    for i in result_charm3: # 매력 포인트 댓글 별로 나누기(두 개 이상 입력한 경우가 있어 적용)
        for j in i:
            if len(j)>2:
                temp = []
                for k in j.findAll('em'):
                    temp.append(k.text)
                charm3.append(temp)
            else:
                charm3.append(j.text)

    from functools import reduce
    result4 = reduce(lambda x,y:x+y,result3) # 댓글 내 리스트 전부 합치기

    df = pd.DataFrame(data=(result4,charm3)).T # 데이터 프레임 생성
    df.columns = ['comment', 'charm']
    
    global df_dict
    df_dict = dict()
    
    df_dict[name]=df

In [50]:
def show_movie():
    driver = webdriver.Chrome('chromedriver')
    url = 'https://www.megabox.co.kr/booking/timetable'
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    movie = soup.select('#mCSB_2_container > ul')
    m = movie[0].select('button.btn')
    
    movie_list=[]
    for i in range(0,22):
        movie_list.append(m[i].text)
    
    global df_movie_list_megabox
    df_movie_list_megabox = pd.DataFrame([i for i in movie_list])
    df_movie_list_megabox.columns = ['name']
    
    return df_movie_list_megabox

In [51]:
def find_url():
    num = int(input('영화의 인덱스를 입력해주세요 : '))
    
    global name
    name = df_movie_list_megabox.loc[num]['name']
    
    driver = webdriver.Chrome('chromedriver')
    url = 'https://www.megabox.co.kr/booking/timetable'
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    mv = soup.select('#mCSB_2_container > ul')
    
    global f_url
    f_url = 'https://www.megabox.co.kr/movie-detail/comment?rpstMovieNo=' + mv[0].select('button.btn')[num]['data-movie-no']
    
    return None

In [52]:
show_movie()

,name
0,컴백홈
1,인생은 아름다워
2,스마일
3,[워너필소] 블레이드 러너: 파이널 컷
4,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교
5,공조2: 인터내셔날
6,고고다이노 극장판: 얼음공룡의 비밀
7,정직한 후보2
8,스파이더맨: 노 웨이 홈 '펀 버전'
9,은하철도 999 - 극장판


In [53]:
find_url()

영화의 인덱스를 입력해주세요 : 4


In [57]:
find_review()